In [94]:
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d


"""
Search for Giles & Deservire Modelling considers the field overlap

"""
class planar_waveguide:
    """
    Calculates the parameters for a planar waveguide
    input:
    n_1 = cladding refractive index
    n_2 = core refractive index
    n_3 = cladding refractive index
    eps_rel_1 = Relative Permittivity cladding (1)
    eps_rel_1 = Relative Permittivity of the core (2)
    eps_rel_3 = Relative Permittivity cladding (3) 
    wavelength = wavelength in nanometers
    W = width in microns
    steps = Number of steps for the iteration
    q = Number of modes to be searched for (starting from the fundamental mode 0)

    
    """
    
    def __init__(self,*,n_1 = 1.455, n_2 = 1.600, n_3 = 1.455, eps_rel_1 = 1, eps_rel_2 = 1, eps_rel_3 = 1,  
                 wavelength = 1550, W = 1.1, steps = 1e6, q = 7 ):
        def islarger(data1, data2):
            """
            Returns the larger value
            """
            if data1>data2:
                return data1
            else:
                return data2
            
        self.q = np.int64(q)
        self.n_1 = np.float64(n_1)                                 #Refractive index cladding
        self.n_2 = np.float64(n_2)                                 #Refractive index core
        self.n_3 = np.float64(n_3)                                 #Refractive index cladding
        self.larger = islarger(self.n_1,self.n_3)                  #Stores the larger cladding refractive index
        self.wavelength = np.float64(wavelength*1e-9)              #Wavelength in nanometres (converted to metres)
        self.W = np.float64(W*1e-6)                                #Width in micrometres (converted to metres)
        self.steps = np.int64(steps)                               #Number of steps
        self.eps_0 = np.float64(8.854*1e-12)                       #Permittivity of free space in vacuum
        self.mu_0 = np.float64(4*np.pi*1e-7)                       #Permeability of free space in vacuum
        self.c = np.float64(1/np.sqrt(self.eps_0 * self.mu_0))     #Speed of light in m/s
        self.f = np.float64(self.c/self.wavelength)                #Frequency in Hz in vacuum
        self.w_ang = np.float64((2*np.pi)*(self.f))                #Angular velocity rad/s in vacuum
        self.k_0 = self.w_ang / self.c                             #Wavenumber in Vacuum
        self.eps_rel_1 = np.float64(eps_rel_1)                     #Relative Permittivity of cladding 1 (Affects TM mode)
        self.eps_rel_2 = np.float64(eps_rel_2)                     #Relative Permittivity of core (2) (Affects TM mode)        
        self.eps_rel_3 = np.float64(eps_rel_3)                     #Relative Permittivity of cladding 3 (Affects TM mode)
        
        
        
    def TEmode(self):
        """
        Finds the transversal electric mode
        """
        init_matrix = np.zeros(int(self.steps))
        init_modes = np.zeros((self.q,1))
        init_modes = init_modes + np.resize(np.arange(0, self.q, 1),(self.q,1))
        init_matrix = (init_matrix) + np.linspace(self.larger, self.n_2, self.steps, endpoint = False)
        gamma_1 = self.k_0 * np.sqrt(init_matrix**2 - self.n_1**2)
        gamma_2 = self.k_0 * np.sqrt(self.n_2**2 - init_matrix**2)
        gamma_3 = self.k_0 * np.sqrt(init_matrix**2 - self.n_3**2)
        result_matrix = np.absolute(gamma_2 * self.W - np.arctan(gamma_1/gamma_2) - np.arctan(gamma_3/gamma_2) 
                                    - init_modes*np.pi)
        results = OrderedDict()
        i = 0
        for elements in result_matrix:
            if np.argmin(elements)!= 0:
                results[i] = init_matrix[np.argmin(elements)]
            elif np.argmin(elements) == 0:
                break
            i += 1
        
        x = np.linspace(0,self.W,self.steps)
        try:
            gamma_1sol = self.k_0 * np.sqrt(results[0]**2 - self.n_1**2)
            gamma_2sol = self.k_0 * np.sqrt(self.n_2**2 - results[0]**2)
            alpha = -np.arctan(gamma_1sol/gamma_2sol) + init_modes * np.pi
            fig = plt.figure()
            plt.subplot(311)
            plt.plot(x, np.cos(gamma_2sol*x + alpha[0]), color = 'b')
            plt.grid(True)
            plt.ticklabel_format(style = 'sci', axis = 'x', scilimits = (0,0))
            plt.subplot(312)
            plt.grid(True)
            plt.ylabel('Intensity of electric field')
            plt.ticklabel_format(style = 'sci', axis = 'x', scilimits = (0,0))
            gamma_1sol = self.k_0 * np.sqrt(results[1]**2 - self.n_1**2)
            gamma_2sol = self.k_0 * np.sqrt(self.n_2**2 - results[1]**2)
            alpha = -np.arctan(gamma_1sol/gamma_2sol) + init_modes * np.pi
            plt.plot(x, np.cos(gamma_2sol*x + alpha[1]), color = 'g')
            plt.subplot(313)
            gamma_1sol = self.k_0 * np.sqrt(results[2]**2 - self.n_1**2)
            gamma_2sol = self.k_0 * np.sqrt(self.n_2**2 - results[2]**2)
            alpha = -np.arctan(gamma_1sol/gamma_2sol) + init_modes * np.pi
            plt.plot(x, np.cos(gamma_2sol*x + alpha[2]), color = 'm')
            plt.grid(True)
            plt.ticklabel_format(style = 'sci', axis = 'x', scilimits = (0,0))
        except:
            pass
        finally:
            plt.xlabel('Width in micrometres')
            plt.show()
    
        return results
        
    def TMmode(self):
        """
        Finds the transversal magnetic mode
        """
        init_matrix = np.zeros(int(self.steps))
        init_modes = np.zeros((self.q,1))
        init_modes = init_modes + np.resize(np.arange(0, self.q, 1),(self.q,1))
        init_matrix = (init_matrix) + np.linspace(self.larger, self.n_2, self.steps, endpoint = False)
        gamma_1 = self.k_0 * np.sqrt(init_matrix**2 - self.n_1**2)
        gamma_2 = self.k_0 * np.sqrt(self.n_2**2 - init_matrix**2)
        gamma_3 = self.k_0 * np.sqrt(init_matrix**2 - self.n_3**2)
        a = self.eps_rel_2 / self.eps_rel_1
        b = self.eps_rel_2 / self.eps_rel_3
        result_matrix = np.absolute(gamma_2 * self.W - np.arctan(a*gamma_1/gamma_2) - np.arctan(b*gamma_3/gamma_2) 
                                    - init_modes*np.pi)
        results = OrderedDict()
        i = 0
        for elements in result_matrix:
            if np.argmin(elements)!= 0:
                results[i] = init_matrix[np.argmin(elements)]
            elif np.argmin(elements) == 0:
                break
            i += 1
        return results   
    
################################################################################################################
        
class rectangular_waveguide:
    """
    Note: n = np.sqrt(eps_rel * mu_rel), if mu_rel is 1 then eps_rel
    a = Width
    b = Depth
    """
    
    def __init__(self,*,n_1 = 1.6, 
                 n_2 = 1.455, 
                 eps_rel_1 = 1, 
                 eps_rel_2 = 1, 
                 wavelength = 980, 
                 a = 0.8, 
                 b = 1.6, 
                 steps = 1e6, 
                 p = 3, 
                 q = 3, 
                 enable_plot = True,
                 show_plot = True 
                ):
        self.n_1 = np.float64(n_1)                                 #Refractive index of core
        self.n_2 = np.float64(n_2)                                 #Refractive index of Cladding
        self.eps_rel_1 = np.float64(eps_rel_1)                     #Relative Permittivity of core
        self.eps_rel_2 = np.float64(eps_rel_2)                     #Relative Permittivity of cladding
        self.wavelength = np.float64(wavelength*1e-9)              #Wavelength in nanometres (converted to metres)
        self.a = np.float64(a*1e-6)                                #Half of the width in micrometres (converted to metres)
        self.b = np.float64(b*1e-6)                                #Half the height in micrometres (converted to metres)
        self.steps = np.int64(steps)                               #Number of steps
        self.p = np.int64(p)                                       #Number of modes in the x direction, starting from 1
        self.q = np.int64(q)                                       #Number of modes in the y direction, starting from 1
        self.eps_0 = np.float64(8.854*1e-12)                       #Permittivity of free space in vacuum
        self.mu_0 = np.float64(4*np.pi*1e-7)                       #Permeability of free space in vacuum
        self.c = np.float64(1/np.sqrt(self.eps_0 * self.mu_0))     #Speed of light in m/s
        self.f = np.float64(self.c/self.wavelength)                #Frequency in Hz in vacuum
        self.w_ang = np.float64((2*np.pi)*(self.f))                #Angular velocity rad/s in vacuum
        self.k_0 = self.w_ang / self.c                             #Wavenumber in Vacuum
        self.enable_plot = enable_plot
        self.show_plot = show_plot
        
        
    def EXmode(self):
        """
        Calculates the electric field polarized in the x direction
        """
        k_0 = self.k_0
        n_1 = self.n_1
        n_2 = self.n_2
        a = self.a
        b = self.b
        steps = self.steps
        limit = np.sqrt(k_0**2 * (n_1**2 - n_2**2))
        pi = np.pi
        p = np.resize(np.arange(1,self.p+1),(self.p,1))
        q = np.resize(np.arange(1,self.q+1),(self.q,1))
        k_x = np.resize(np.linspace(1e-6,limit,steps, endpoint = False),(1,steps))
        gamma_x = np.sqrt(k_0**2 * (n_1**2 - n_2**2) - k_x**2)
        k_y = np.resize(np.linspace(1e-6,limit,steps, endpoint = False),(1,steps))
        gamma_y = np.sqrt(k_0**2 * (n_1**2 - n_2**2) - k_y**2)
        y_sol = np.absolute(k_y * b - np.arctan(gamma_y/k_y) - 0.5 * pi * (q-1))
        i = 1
        y_sols = OrderedDict()
        for elements in y_sol:
            y_sols[i] = {'k_y':k_y[0][np.argmin(elements)], 'gamma_y':gamma_y[0][np.argmin(elements)]}
            i += 1
        
        x_sols = OrderedDict()
        for i in range(1,self.p+1):
            g = (k_0**2 * n_1**2 - y_sols[i]['k_y']**2) * gamma_x
            h = (k_0**2 * n_2**2 - y_sols[i]['k_y']**2) * k_x
            x_sol = np.absolute(k_x * b - np.arctan(g/h) - 0.5 * pi * (i-1))
            x_sols[i] = {'k_x':k_x[0][np.argmin(x_sol)], 'gamma_x':gamma_x[0][np.argmin(x_sol)]}
        
        results = OrderedDict()
        constants = OrderedDict()
        for p, value_x in x_sols.items():
            for q, value_y in y_sols.items():
                k_x = value_x['k_x']
                gamma_x = value_x['gamma_x']
                k_y = value_y['k_y']
                gamma_y = value_y['gamma_y']
                #t = (k_0**2 * n_1**2 - k_y**2) * gamma_x
                #u = (k_0**2 * n_2**2 - k_y**2) * k_x
                #alpha_1 = np.arctan(t/u) + p * pi - k_x * a
                #alpha_2 = np.arctan(gamma_y/k_y) + q * pi - k_y * b
                alpha_1 = (p-1) * np.pi * 0.5
                alpha_2 = (q-1) * np.pi * 0.5
                beta = np.sqrt(k_0**2 * n_1**2 - k_x**2 - k_y**2)
                n_eff = beta / k_0
                #print(p,q,n_eff)
                if n_2 < n_eff <= n_1:
                    results[p,q] = {
                        'k_x':k_x, 'k_y': k_y, 
                        'gamma_x': gamma_x, 'gamma_y': gamma_y, 
                        'alpha_1':alpha_1, 'alpha_2':alpha_2,
                        'beta':beta, 'n_eff':n_eff,
                        }
                    """
                    Note alpha_1 and alpha_2 have the signs reversed compared to the book to be compatible with the other
                    book and the E_x determination later on
                    """ 
                    c_1 = 1   #Initial value of constant Corresponding to intensity
                    c_2 = c_1 #* k_y * np.sin(k_y * b + alpha_2) / gamma_y
                    c_3 = c_1 #* k_x * np.sin(k_x * a + alpha_1) / gamma_x
                    c_4 = c_1 #* k_y * np.sin(k_y * b - alpha_2) / gamma_y
                    c_5 = c_1 #* k_x * np.sin(k_x * a - alpha_1) / gamma_x                    
                    """
                    c_2 = c_1 * k_y * np.sin(k_y * b + alpha_2) / gamma_y
                    c_3 = c_1 * k_x * np.sin(k_x * a + alpha_1) / gamma_x
                    c_4 = c_1 * k_y * np.sin(k_y * b - alpha_2) / gamma_y
                    c_5 = c_1 * k_x * np.sin(k_x * a - alpha_1) / gamma_x
                    """
                    constants[p,q] = {
                        'c_1':c_1,
                        'c_2':c_2,
                        'c_3':c_3,
                        'c_4':c_4,
                        'c_5':c_5
                        }
        for p,q in results.keys():
            if self.enable_plot == False:
                break
            modep = p
            modeq = q
            #Color bar plot
            #Region 1
            x = np.linspace(-a, a, 100)
            y = np.linspace(-b, b, 100)
            x, y = np.meshgrid(x,y)
            E_x = constants[modep,modeq]['c_1'] * np.cos(results[modep,modeq]['k_x'] * x -\
            results[modep,modeq]['alpha_1']) * np.cos(results[modep,modeq]['k_y'] * y -\
            results[modep,modeq]['alpha_2'])
            #Region 2
            x2 = np.linspace(-a, a, 100)
            y2 = np.linspace(b, 3 * b, 100)
            x2, y2 = np.meshgrid(x2,y2)
            E_x2 = constants[modep,modeq]['c_2'] * np.cos(results[modep,modeq]['k_x'] * x2 -\
            results[modep,modeq]['alpha_1']) * np.exp(-results[modep,modeq]['gamma_y'] * (y2 - b)) *\
            np.cos(results[modep,modeq]['k_y'] * b - results[modep,modeq]['alpha_2'])
            #Region 3
            x3 = np.linspace(a, 3 * a, 100)
            y3 = np.linspace(-b, b, 100)
            x3, y3 = np.meshgrid(x3,y3)
            E_x3 = constants[modep,modeq]['c_3'] * np.cos(results[modep,modeq]['k_x'] *\
            a - results[modep,modeq]['alpha_1']) * np.exp(-results[modep,modeq]['gamma_x'] * (x3 - a)) *\
            np.cos(results[modep,modeq]['k_y'] * y3 - results[modep,modeq]['alpha_2'])
            #Region 4
            x4 = np.linspace(-a, a, 100)
            y4 = np.linspace(-b, -3 * b, 100)
            x4, y4 = np.meshgrid(x4,y4)
            E_x4 = constants[modep,modeq]['c_4'] * np.cos(results[modep,modeq]['k_x'] * x4 +\
            results[modep,modeq]['alpha_1']) * np.exp(results[modep,modeq]['gamma_y'] * (y4 + b))
            #Region 5
            x5 = np.linspace(-a,-3 * a, 100)
            y5 = np.linspace(-b, b, 100)
            x5, y5 = np.meshgrid(x5,y5)
            E_x5 = constants[modep,modeq]['c_5'] * np.exp(results[modep,modeq]['gamma_x'] * (x5 + a)) *\
            np.cos(results[modep,modeq]['k_y'] * y5 + results[modep,modeq]['alpha_2'])
            
            plt.hold(True)
            z = E_x
            z2 = E_x2
            z3 = E_x3
            z4 = E_x4
            z5 = E_x5
            fig, ax = plt.subplots()
            im = ax.contourf(x,y,z, vmin = -1, vmax = 1, levels = np.linspace(-1,1,100))
            #ima = ax.contour(x, y, z, colors='k')
            im2 = ax.contourf(x2, y2, z2, vmin = -1, vmax = 1)
            #im2a = ax.contour(x2, y2, z2, colors='k')
            im3 = ax.contourf(x3, y3, z3, vmin = -1, vmax = 1)
            #im3a = ax.contour(x3, y3, z3, colors='k')
            im4 = ax.contourf(x4, y4, z4, vmin = -1, vmax = 1)
            #im4a = ax.contour(x4, y4, z4, colors='k')
            im5 = ax.contourf(x5, y5, z5, vmin = -1, vmax = 1)
            #im5a = ax.contour(x5, y5, z5, colors='k')
            plt.xlabel('Width (a)')
            plt.ylabel('Depth (b)')
            ax.axhline(y=-b, xmin=0, xmax=1, linewidth=0.8, color = 'k')
            ax.axhline(y=b, xmin=0, xmax=1, linewidth=0.8, color = 'k')
            ax.axvline(x=-a, ymin=0, ymax = 1, linewidth=0.8, color='k')
            ax.axvline(x=a, ymin=0, ymax = 1, linewidth=0.8, color='k')
            fig.colorbar(im, ax=ax, label = 'Electric field intensity')
            plt.ticklabel_format(style = 'sci', axis = 'x', scilimits = (0,0))
            plt.ticklabel_format(style = 'sci', axis = 'y', scilimits = (0,0))
            try:
                os.makedirs(os.getcwd()+"\\plots")
            except:
                pass
            plt.savefig(os.getcwd()+"\\plots\\EX_flat_{}{}.png".format(modep,modeq))
            plt.close("all")
            #Surface plot
            
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.set_xlabel('Width (a)')
            ax.set_ylabel('Depth (b)')
            #ax.set_zlabel('Electric field intensity')
            plt.ticklabel_format(style = 'sci', axis = 'x', scilimits = (0,0))
            plt.ticklabel_format(style = 'sci', axis = 'y', scilimits = (0,0))
            ax.plot_surface(x, y, z, cmap = cm.jet, vmin = -1, vmax = 1, rstride=5, cstride=5, linewidth = 0.1)
            ax.plot_surface(x2, y2, z2, cmap = cm.jet, vmin = -1, vmax = 1, rstride=5, cstride=5, linewidth = 0.1)#, cmap = cm.jet, rstride=10, cstride=10, linewidth = 0.1)
            ax.plot_surface(x3, y3, z3, cmap = cm.jet, vmin = -1, vmax = 1, rstride=5, cstride=5, linewidth = 0.1)
            ax.plot_surface(x4, y4, z4, cmap = cm.jet, vmin = -1, vmax = 1, rstride=5, cstride=5, linewidth = 0.1)
            ax.plot_surface(x5, y5, z5, cmap = cm.jet, vmin = -1, vmax = 1, rstride=5, cstride=5, linewidth = 0.1)
            fig.colorbar(im, ax=ax, label = 'Electric field intensity')
            plt.savefig(os.getcwd()+"\\plots\\EX_surface_{}{}.png".format(modep,modeq))
            
            if self.show_plot == True:
                plt.show()
            plt.close("all")
            
        
        return constants, results

h = rectangular_waveguide()
h.enable_plot = True
h.show_plot = False
w = h.EXmode()
print("done")
        #Note: Note Region 1, 2, 3 modified to be equivalent to the book fundamental of optical waveguides

done


In [95]:
import pprint
h = rectangular_waveguide()
h.enable_plot = True
h.show_plot = False
w = h.EXmode()
print(pprint.pformat(w))

(OrderedDict([((1, 1), {'c_1': 1, 'c_2': 1, 'c_3': 1, 'c_4': 1, 'c_5': 1}),
              ((1, 2), {'c_1': 1, 'c_2': 1, 'c_3': 1, 'c_4': 1, 'c_5': 1}),
              ((1, 3), {'c_1': 1, 'c_2': 1, 'c_3': 1, 'c_4': 1, 'c_5': 1}),
              ((2, 1), {'c_1': 1, 'c_2': 1, 'c_3': 1, 'c_4': 1, 'c_5': 1}),
              ((2, 2), {'c_1': 1, 'c_2': 1, 'c_3': 1, 'c_4': 1, 'c_5': 1}),
              ((2, 3), {'c_1': 1, 'c_2': 1, 'c_3': 1, 'c_4': 1, 'c_5': 1}),
              ((3, 1), {'c_1': 1, 'c_2': 1, 'c_3': 1, 'c_4': 1, 'c_5': 1}),
              ((3, 2), {'c_1': 1, 'c_2': 1, 'c_3': 1, 'c_4': 1, 'c_5': 1}),
              ((3, 3), {'c_1': 1, 'c_2': 1, 'c_3': 1, 'c_4': 1, 'c_5': 1})]),
 OrderedDict([((1, 1),
               {'alpha_1': 0.0,
                'alpha_2': 0.0,
                'beta': 10185027.679966174,
                'gamma_x': 4176589.4768032269,
                'gamma_y': 4180551.9691095636,
                'k_x': 874717.07033627352,
                'k_y': 855578.66058900452,
   

In [71]:
from lmfit import minimize, Parameters, Parameter, fit_report

OrderedDict([(1, 0.000244140625),
             (2, 0.0),
             (3, 0.0),
             (4, 0.0),
             (5, 0.0),
             (6, 0.0)])
